#  Example 1  

For every customer, show the number of orders they have placed for a product whose named contains _helmet_. Name the column containing the product count `Number of Helmet Orders`. For those customers who have never placed such an order, display a value of 0.

**Method 1**:  Without an outer join

In [1]:
use SalesOrdersExample;

--  Strategy:  
--  Find orders containing helmets and count them up by customer
--  UNION
--  Find customers who don't have any orders containing helmets using a NOT EXISTS subquery  (could do EXCEPT, but requires CustomerID column to be 100% correct)
select CustFirstName, CustLastName, count (distinct Orders.OrderNumber) as 'Number of Helmet Orders'
from Orders
join Order_Details
  on Orders.OrderNumber = Order_Details.OrderNumber
join Products
  on Products.ProductNumber = Order_Details.ProductNumber
join Customers
  on Customers.CustomerID = Orders.CustomerID
where ProductName like '%Helmet%'
group by Customers.CustomerID, CustFirstName, CustLastName
UNION
select CustFirstName, CustLastName, 0
from Customers
where not exists (
  select *
  from Orders
  join Order_Details
  on Orders.OrderNumber = Order_Details.OrderNumber
  join Products
  on Products.ProductNumber = Order_Details.ProductNumber
   where Customers.CustomerID = Orders.CustomerID
   and ProductName like '%helmet%'
)


(28 rows affected)

Total execution time: 00:00:00.102

CustFirstName,CustLastName,Number of Helmet Orders
Alaina,Hallmark,0
Andrew,Cencini,11
Angel,Kennedy,12
Caleb,Viescas,8
Darren,Gehring,10
David,Smith,8
Dean,McCrae,21
Estella,Pundt,15
Gary,Hallmark,8
Jeffrey,Tirekicker,0


Repeat the above method but sort by the number of orders of helmets.   You can only have an `ORDER BY` clause on the final query in a `UNION` query.  It applies to the results **as a whole**.

In [2]:
use SalesOrdersExample;

--  Strategy:  
--  Find orders containing helmets and count them up by customer
--  UNION
--  Find customers who don't have any orders containing helmets using a NOT EXISTS subquery  (could do EXCEPT, but requires CustomerID column to be 100% correct)
select CustFirstName, CustLastName, count (distinct Orders.OrderNumber) as 'Number of Helmet Orders'
from Orders
join Order_Details
  on Orders.OrderNumber = Order_Details.OrderNumber
join Products
  on Products.ProductNumber = Order_Details.ProductNumber
join Customers
  on Customers.CustomerID = Orders.CustomerID
where ProductName like '%Helmet%'
group by Customers.CustomerID, CustFirstName, CustLastName
UNION
select CustFirstName, CustLastName, 0
from Customers
where not exists (
  select *
  from Orders
  join Order_Details
  on Orders.OrderNumber = Order_Details.OrderNumber
  join Products
  on Products.ProductNumber = Order_Details.ProductNumber
   where Customers.CustomerID = Orders.CustomerID
   and ProductName like '%helmet%'
)
order by [Number of Helmet Orders] desc

(28 rows affected)

Total execution time: 00:00:00.064

CustFirstName,CustLastName,Number of Helmet Orders
Dean,McCrae,21
Rachel,Patterson,17
Estella,Pundt,15
Robert,Brown,15
Liz,Keyser,13
Neil,Patterson,13
William,Thompson,13
Angel,Kennedy,12
Manuela,Seidel,12
Mariya,Sergienko,12


In [3]:
use SalesOrdersExample
-- Quick check to make sure the above is right
select count(*) from Customers

(1 row affected)

Total execution time: 00:00:00.009

(No column name)
28


Now try it using an outer join the "straightforward" way.

In [4]:
use [SalesOrdersExample];

select CustFirstName, CustLastName, count (distinct Orders.OrderNumber) as 'Number of Helmet Orders'
from Orders
join Order_Details
  on Orders.OrderNumber = Order_Details.OrderNumber
join Products
  on Products.ProductNumber = Order_Details.ProductNumber
right join Customers
  on Orders.CustomerID = Customers.CustomerID
where ProductName like '%Helmet%'
group by Customers.CustomerID, CustFirstName, CustLastName
order by [Number of Helmet Orders] desc;

(25 rows affected)

Total execution time: 00:00:00.024

CustFirstName,CustLastName,Number of Helmet Orders
Dean,McCrae,21
Rachel,Patterson,17
Robert,Brown,15
Estella,Pundt,15
Liz,Keyser,13
William,Thompson,13
Neil,Patterson,13
Suzanne,Viescas,12
Mariya,Sergienko,12
Angel,Kennedy,12


This doesn't work because:

- the `FROM` clause is evaluated **before** the `WHERE` clause.
- the `right join` causes all `Customer` tuples to be included, with `null` values for any Customers who don't join to any `ORDERS`
- any comparison with `null` is **false**

Let's take a look at the orders for the 3 customers who showed 0 results for orders containing _Helmet_ in more detail. They are:

- Alaina Hallmark (there's also a **Gary** Hallmark)
- Julia Schnebly
- Jeffrey Tireticker

In [5]:
use [SalesOrdersExample];

select distinct CustFirstName, CustLastName, ProductName
from Orders
join Order_Details
  on Orders.OrderNumber = Order_Details.OrderNumber
join Products
  on Products.ProductNumber = Order_Details.ProductNumber
right join Customers
  on Customers.CustomerID = Orders.CustomerID
where CustLastName in ('Tirekicker', 'Schnebly') or (CustFirstName = 'Alaina' and CustLastName = 'Hallmark')
order by CustLastName, ProductName

(62 rows affected)

Total execution time: 00:00:00.039

CustFirstName,CustLastName,ProductName
Alaina,Hallmark,AeroFlo ATB Wheels
Alaina,Hallmark,Clear Shade 85-T Glasses
Alaina,Hallmark,Cosmic Elite Road Warrior Wheels
Alaina,Hallmark,Cycle-Doc Pro Repair Stand
Alaina,Hallmark,Dog Ear Aero-Flow Floor Pump
Alaina,Hallmark,Dog Ear Monster Grip Gloves
Alaina,Hallmark,Eagle FS-3 Mountain Bike
Alaina,Hallmark,Eagle SA-120 Clipless Pedals
Alaina,Hallmark,GT RTS-2 Mountain Bike
Alaina,Hallmark,HP Deluxe Panniers


So the `WHERE` clause eliminates _Jeffrey Tirekicker_ because the value of `ProductName` is `null`, since he has never placed any orders. The `WHERE` clause eliminates _Alaina Hallmark_ and _Julia Schnebly_ since **none** of their orders include a product whose name contains _helmet_.

**Method 2:**  To get the correct answer using an outer join, we have to find the orders containing helmets **first**, and then do the outer join to **only those orders**. The easiest way to accomplish this is to use a common table expression.

In [3]:
use [SalesOrdersExample];

with HelmetOrders as (
    select CustomerID, Orders.OrderNumber, ProductName
    from Orders
    join Order_Details
      on Orders.OrderNumber = Order_Details.OrderNumber
    join Products
      on Products.ProductNumber = Order_Details.ProductNumber
    where ProductName like '%Helmet%'
)
select CustFirstName, CustLastName, count(distinct HelmetOrders.OrderNumber) 'Number of Helmet Orders'
from HelmetOrders 
right join Customers
  on HelmetOrders.CustomerID = Customers.CustomerID
group by Customers.CustomerID, CustFirstName, CustLastName
order by 'Number of Helmet Orders' desc

Warning: Null value is eliminated by an aggregate or other SET operation.

(28 rows affected)

Total execution time: 00:00:00.059

CustFirstName,CustLastName,Number of Helmet Orders
Dean,McCrae,21
Rachel,Patterson,17
Robert,Brown,15
Estella,Pundt,15
William,Thompson,13
Neil,Patterson,13
Liz,Keyser,13
Mariya,Sergienko,12
Angel,Kennedy,12
Suzanne,Viescas,12


As a general rule, do any filtering on tables that are being inner joined in a query containing an outer join in a CTE **before** performing the outer join.

# Example 2

In the `SchoolSchedulingExample`, use an outer join along with a `WHERE` clause to list all courses that don't appear on any student's schedule.

In [2]:
use [SchoolSchedulingExample];
select * from Student_Schedules
right join Classes
    on Student_Schedules.ClassID = Classes.ClassID


(220 rows affected)

Total execution time: 00:00:00.047

StudentID,ClassID,ClassStatus,Grade,ClassID,SubjectID,ClassRoomID,Credits,SemesterNumber,StartDate,StartTime,Duration,MondaySchedule,TuesdaySchedule,WednesdaySchedule,ThursdaySchedule,FridaySchedule,SaturdaySchedule
1001,1000,2,99.83,1000,11,1231,5,1,2017-09-12,10:00:00,50,0,1,1,1,1,1
1005,1000,2,82.19,1000,11,1231,5,1,2017-09-12,10:00:00,50,0,1,1,1,1,1
1006,1000,2,73.04,1000,11,1231,5,1,2017-09-12,10:00:00,50,0,1,1,1,1,1
1008,1000,2,89.32,1000,11,1231,5,1,2017-09-12,10:00:00,50,0,1,1,1,1,1
1012,1000,2,71.15,1000,11,1231,5,1,2017-09-12,10:00:00,50,0,1,1,1,1,1
1013,1000,2,69.67,1000,11,1231,5,1,2017-09-12,10:00:00,50,0,1,1,1,1,1
1017,1000,2,69.46,1000,11,1231,5,1,2017-09-12,10:00:00,50,0,1,1,1,1,1
NULL,NULL,NULL,NULL,1002,12,1619,4,1,2017-09-11,15:30:00,110,1,0,1,0,0,0
NULL,NULL,NULL,NULL,1004,13,1627,4,1,2017-09-11,08:00:00,50,1,0,1,1,1,0
NULL,NULL,NULL,NULL,1006,13,1627,4,1,2017-09-11,09:00:00,110,1,0,1,0,0,0


# Example 3
List the `RecipeClassDescription` for **all** recipe classes, along with the titles of any recipes belonging to that class.

In [4]:
use [RecipesExample];
select RecipeClassDescription, RecipeTitle from Recipe_Classes
left join Recipes on Recipe_Classes.RecipeClassID = Recipes.RecipeClassID


(16 rows affected)

Total execution time: 00:00:00.048

RecipeClassDescription,RecipeTitle
Main course,Irish Stew
Main course,Fettuccini Alfredo
Main course,Pollo Picoso
Main course,Roast Beef
Main course,"Huachinango Veracruzana (Red Snapper, Veracruz style)"
Main course,Tourtière (French-Canadian Pork Pie)
Main course,Salmon Filets in Parchment Paper
Vegetable,Garlic Green Beans
Vegetable,Asparagus
Starch,Yorkshire Pudding


Modify the previous query to use the SQL Server [CASE statement](https:\link.hope.edu\392-sql-case) to display the string **No Recipes For This Class** for the title for any recipe classes for which there are no recipes

In [6]:
use [RecipesExample];
select RecipeClassDescription,
CASE
    WHEN RecipeTitle is NULL THEN 'No Recipes For This Class'
    else RecipeTitle
    END as RecipeTitle
from Recipe_Classes
left join Recipes on Recipe_Classes.RecipeClassID = Recipes.RecipeClassID 




(16 rows affected)

Total execution time: 00:00:00.032

RecipeClassDescription,RecipeTitle
Main course,Irish Stew
Main course,Fettuccini Alfredo
Main course,Pollo Picoso
Main course,Roast Beef
Main course,"Huachinango Veracruzana (Red Snapper, Veracruz style)"
Main course,Tourtière (French-Canadian Pork Pie)
Main course,Salmon Filets in Parchment Paper
Vegetable,Garlic Green Beans
Vegetable,Asparagus
Starch,Yorkshire Pudding


# Practice problems

Using an outer join, find all products that have never been ordered.  (2 products, _Victoria Pro All Weather Tires_ and _Ultra-Pro Rain Jacket_)

In [10]:
use SalesOrdersExample;
select ProductName from Products
left join Order_Details on Products.ProductNumber = Order_Details.ProductNumber
where OrderNumber is NULL


(2 rows affected)

Total execution time: 00:00:00.062

ProductNumber,ProductName,ProductDescription,RetailPrice,QuantityOnHand,CategoryID,OrderNumber,ProductNumber,QuotedPrice,QuantityOrdered
23,Ultra-Pro Rain Jacket,NULL,85.00,30,3,NULL,NULL,NULL,NULL
4,Victoria Pro All Weather Tires,NULL,54.95,20,4,NULL,NULL,NULL,NULL


Find all products that weren't ordered in the first 6 months of 2017. Use the `month` and `year` functions to get the appropriate parts of the `OrderDate` column. Sort the orders by product name (40 products, first is _AeroFlo ATB Wheels_ and last is _X-Pro All Weather Tires_)

In [21]:
use SalesOrdersExample;
select Products.ProductNumber, ProductName from Products
left join Order_Details on Products.ProductNumber = Order_Details.ProductNumber
left join Orders on Orders.OrderNumber = Order_Details.OrderNumber
where MONTH(OrderDate) > 6 and YEAR(OrderDate) = 2017
order by ProductName ASC

(38 rows affected)

Total execution time: 00:00:00.048

ProductNumber,ProductName
37,AeroFlo ATB Wheels
30,Clear Shade 85-T Glasses
36,Cosmic Elite Road Warrior Wheels
38,Cycle-Doc Pro Repair Stand
21,Dog Ear Aero-Flow Floor Pump
3,Dog Ear Cyclecomputer
5,Dog Ear Helmet Mount Mirrors
20,Dog Ear Monster Grip Gloves
2,Eagle FS-3 Mountain Bike
14,Eagle SA-120 Clipless Pedals


Display the names of all customers and the number of orders they have placed that included a _mountain bike_ in 2018.  

Sort the results so that the customers placing the most orders for mountain bikes appear first.  (28 customers, with _Dean McCrae_ and _Luke Patterson_ both having placed **14** orders for mountain bikes, and 5 customers having **0** orders, including _Angel Kennedy_ and _Jeffrey Tirekicker)._

In [ ]:
use SalesOrdersExample;